In [1]:
from langchain_community.chat_models import ChatOllama
from langchain_community.embeddings import OllamaEmbeddings
from langchain_aws import ChatBedrock
from langfuse.callback import CallbackHandler
import asyncio
import nest_asyncio

nest_asyncio.apply()
from dotenv import load_dotenv
import os

load_dotenv()

HAIKU_ID = "anthropic.claude-3-haiku-20240307-v1:0"
SONNET_ID = "anthropic.claude-3-sonnet-20240229-v1:0"

llm = ChatOllama(model="mistral:v0.3", temperature=0.2)
bedrock_chat = ChatBedrock(
    model_id=SONNET_ID,
    model_kwargs={"temperature": 0.1},
)
embedding_function = OllamaEmbeddings(model="nomic-embed-text")

secret_key = os.getenv("SECRET_KEY")
public_key = os.getenv("PUBLIC_KEY")
langfuse_handler = CallbackHandler(
    secret_key=os.getenv("SECRET_KEY"),
    public_key=os.getenv("PUBLIC_KEY"),
    host="https://us.cloud.langfuse.com",  # 🇺🇸 US region
)

In [3]:
import os
import sys

nb_dir = os.path.split(os.getcwd())[0]
if nb_dir not in sys.path:
    sys.path.append(nb_dir)
from ingestion.ckan_loader import CKanLoader

loader = CKanLoader()

CKanLoader initialized, found 1453 dataset ids.


In [ ]:

# from langchain_community.vectorstores import Chroma
# db = Chroma.from_documents(
#     documents=loader.load(),
#     embedding=embedding_function,
#     persist_directory="./chroma_db",
#     collection_name="robco_chroma"
# )

In [4]:
from langchain_community.vectorstores import Chroma

vectorstore = Chroma(
    persist_directory="./chroma_db",
    embedding_function=embedding_function,
    collection_name="robco_chroma"
)

print("There are", vectorstore._collection.count(), "docs in the collection")
vectorstore._collection.name

There are 1454 docs in the collection


'robco_chroma'

In [14]:
from langchain_core.prompts import PromptTemplate
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

retriever = vectorstore.as_retriever(
    search_type="mmr",
    search_kwargs={
        "k": 5,
        "lambda_mult": 0.5,
    },
)

def format_docs(docs):
    return "\n\n".join([d.page_content for d in docs])

prompt = PromptTemplate.from_template("Based on those documents <documents>{documents}</documents> answer the user query <query>{query}</query>")
chain = retriever | prompt | bedrock_chat
chain = (
    {"documents": retriever | format_docs, "query": RunnablePassthrough()}
    | prompt
    | bedrock_chat
    | StrOutputParser()
)
res = chain.invoke(input={"query":"avez vous des jeux de donnés sur les bibliothèques ?"},config={"callbacks": [langfuse_handler]})
res

'D\'après les documents fournis, je ne trouve pas de jeu de données spécifique sur les bibliothèques. Les jeux de données semblent porter sur des sujets comme l\'imagerie aérienne, les travaux en cours, les points de rencontre d\'ambulance, la cartographie forestière et les parcs municipaux avec leurs services. Cependant, il est mentionné pour le jeu de données "Parcs municipaux avec services - Ville de Shawinigan" qu\'il y a un attribut "service_boitelivre" qui pourrait indiquer la présence de boîtes à livres dans certains parcs. Mais il n\'y a pas de données détaillées sur les bibliothèques municipales ou autres types de bibliothèques.'

In [ ]:
from langchain.retrievers.self_query.base import SelfQueryRetriever

s_retriever = SelfQueryRetriever.from_llm(
    llm=bedrock_chat,
    vectorstore=vectorstore,
    search_type="mmr",
    document_contents=loader.document_content_info,
    metadata_field_info=loader.metadata_field_info,
    search_kwargs={
        "k": 5,
    },
    
)
res = s_retriever.invoke(
    "Avez vous des jeux de donnés avec des ressources pdf ?",
    config={"callbacks": [langfuse_handler]},
)
res